#Лабораторная работа №3. Web-scraping

# Часть 1. Парсинг сайта через библиотеки requests и Beautifulsoup и lxml

In [ ]:
import requests

url = 'https://www.imdb.com/search/title/?groups=top_1000&count=250'
r = requests.get(url)
r.content

In [ ]:
s=str(r.text.encode('utf-8'))
with open('test.html', 'w') as output_file:
  output_file.write(s)

In [ ]:
header= {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
'accept-encoding': 'gzip, deflate, br',
'accept-language': 'en,en-US;q=0.9',
'cache-control': 'max-age=0',
'referer': 'https://www.imdb.com/search/title/',
'upgrade-insecure-requests': '1',
'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36'
}

In [ ]:
url = 'https://www.imdb.com/search/title/?groups=top_1000&count=250'
r = requests.get(url, headers=header, timeout=(3.05, 27))
r.content

In [ ]:
with open('test2.html', 'w') as output_file:
  output_file.write(str(r.text.encode('utf-8')))

In [ ]:
#XPath — it is a language for querying xml and xhtml documents.
from lxml import html
test = '''
    <html>
        <body>
            <div class="first_level">
                <h2 align='center'>one</h2>
                <h2 align='left'>two</h2>
            </div>
            <h2>another tag</h2>
        </body>
    </html>
'''
tree = html.fromstring(test)
tree.xpath('//h2') # все h2 теги
tree.xpath('//h2[@align]') # h2 теги с атрибутом align
tree.xpath('//h2[@align="center"]') # h2 теги с атрибутом align равным "center"

div_node = tree.xpath('//div')[0] # div тег
div_node.xpath('.//h2') # все h2 теги, которые являются дочерними div ноде

In [ ]:
from bs4 import BeautifulSoup
from lxml import html

# Beautiful Soup
soup = BeautifulSoup(r.text)
film_list = soup.find('div', {'class': 'lister-list'})

# lxml 
tree = html.fromstring(r.text)
film_list_lxml = tree.xpath('//div[@class = "lister-list"]')[0]

In [ ]:
film_list

In [ ]:
# Beatiful Soup
items = film_list.find_all('h3', {'class': ['lister-item-header']})
names_films=[item.find('a').text for item in items]


# lxml
items_lxml = film_list_lxml.xpath('//h3[@class = "lister-item-header"]')
names_films_lxml = [item_lxml.xpath('.//a/text()')[0] for item_lxml in items_lxml]

names_films_lxml

In [ ]:
# Beatiful Soup
items = film_list.find_all('div', {'class': ['ratings-bar']})
strong_films=[item.find('div', {'class':'inline-block ratings-imdb-rating'}).find('strong').text for item in items]


# lxml
items_lxml = film_list_lxml.xpath('//div[@class = "ratings-bar"]')
strong_films_lxml = [item_lxml.xpath('.//div[@class = "inline-block ratings-imdb-rating"]/strong/text()')[0] for item_lxml in items_lxml]

strong_films_lxml

In [ ]:
dictionary_films = dict(zip(names_films, strong_films))
dictionary_films

## **Задание 1**
1. С сайта https://www.kinopoisk.ru/lists/movies/popular-films/ получить информацию в csv-файл о всех фильмах с помощью библиотек BeatifulSoup и lxml:


*   Название фильма
*   Год
*   продолжительность (мин.)
*   страна
*   жанр фильма
*   актеры
*   рейтинг
*   режиссёр
2. Предусмотреть сбор информации со всех страниц















# Часть 2. Использование библиотеки selenium

In [ ]:
# импортим библиотеки
import requests
import time
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

In [ ]:
# Ставим библиотеку selenium - с помощью неё сымитируем нажатие кнопки и прокрутку
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.common.by import By
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
# Запускаем хром
wd = webdriver.Chrome('chromedriver',options=chrome_options) 
wd.get("https://ria.ru/location_Arkhangelskaja_oblast/")
wd.implicitly_wait(10)
wd.fullscreen_window
print(wd.title) # проверяем работу

In [ ]:
def parse():
  soup = BeautifulSoup(wd.page_source, features="lxml") # создание объекта BeautifulSoup для работы с DOM моделью. Используем данные с selenium
  location_blocks = soup.findAll('div', {'class': 'list-item'}) # получение всех div с классом 'list-item', который содержит всю необходимую информацию
  for block in location_blocks:
        link = []
        title = []
        publdate = []
        vievCount = []
        tags = []
        tagsAll = []
        data = []

        title.append(block.find('div').find('a', {'class': 'list-item__title color-font-hover-only'}).text) # заголовок

        link.append(block.find('a', {'class': 'list-item__title color-font-hover-only'}).get('href')) # Ссылка

        publdate.append(block.find('div', {'class': 'list-item__info'}).find('div', {'class' : 'list-item__date'}).text) # дата публикации

        vievCount.append(block.find('div', {'class': 'list-item__info'}).find('div', {'class' : 'list-item__views'}).find('div', {'class' : 'list-item__views-text'}).text) # количество просмотров
      
        tagsAll = block.find('ul') # берем весь список тегов
        for itemLi in tagsAll:
          tags.append(itemLi.find('a').text)    # раскидываем каждый элемент
        
        data.append(title)
        data.append(link)
        data.append(publdate)
        data.append(vievCount)
        data.append(tags)
        with open('harvest_data.csv',  mode="a", encoding='utf-8-sig') as f: 
          writer = csv.writer(f)
          writer.writerow(data)


wait = WebDriverWait(wd, 30)
wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'list-more'))) # Ждем пока кнопка "Показать больше" будет готова

x = 0
while x<2000:         # создаем длинный цикл, чтобы проявить статьи вызывающиеся по кнопке "Показать больше"
  time.sleep(0.5)
  wd.execute_script('document.querySelector("#content > div > div.layout-rubric__main > div > div.list-more").click()') # Имитируем нажатие на кнопку исполняя скрипт с указанием click()
  x = x+10

time.sleep(2) #немного ждем, чтобы не потерять последние статьи
parse()

In [ ]:
wd.quit()

##**Задание 2**

Спарсить новости с сата https://www.news29.ru/novosti/ekonomika/ или https://www.pomorie.ru/news/ (на выбор студента) с помощью selenium имитируя нажатие кнопки.

# Часть 3. Краулим сайт с помощью Scrappy

Скраулить с сайта http://quotes.toscrape.com информацию о цитатах известных людей

In [ ]:
!git clone https://github.com/harrywang/scrapy-tutorial-starter.git
!cd scrapy-tutorial-starter

In [ ]:
!pip install Scrapy

In [ ]:
!scrapy startproject my_first_spider

In [ ]:
!cd my_first_spider

Первая попытка
```
# # import scrapy
# class QuotesSpider(scrapy.Spider):
#     name = "quotes"
# start_urls = ['http://quotes.toscrape.com']
# def parse(self, response):
#         self.logger.info('hello this is my first spider')
#         pass
```



!scrapy shell http://quotes.toscrape.com/ #запускаем Паука

Пробуем в командной строке получать информацию


```
quotes = response.xpath("//div[@class='quote']")
quotes[0].css(".text::text").getall()
quotes[0].css(".author::text").getall()
quotes[0].css(".tag::text").getall()
```



In [ ]:
!scrapy shell http://quotes.toscrape.com/ #запускаем Паука

Теперь напишем паука


```
# import scrapy

# class QuotesSpider(scrapy.Spider):
#   name = "quotes"
#   start_urls = ['http://quotes.toscrape.com']

#   def parse(self, response):
#     self.logger.info('hello this is my first spider')
#     quotes = response.css('div.quote')
#     for quote in quotes:
#         yield {
#             'text': quote.css('.text::text').get(),
#             'author': quote.css('.author::text').get(),
#             'tags': quote.css('.tag::text').getall(),
#         }
#     next_page = response.css('li.next a::attr("href")').get()
#     cnt = 0
#     if next_page is not None or cnt<10:
#       cnt+=1
#       yield response.follow(next_page, self.parse)
```



In [ ]:
!scrapy runspider /content/my_first_spider/my_first_spider/spiders/quotes-spider.py -o quotes.json

 #запускаем обновленного Паука

In [ ]:
!less quotes.json

## **Задание 3**

Создайте паука для краулинга информации с бесконечно прокручивающийся страницы (http://spidyquotes.herokuapp.com/scroll)